In [26]:
import os
import sys
sys.path.append('/scratch/programming/preprocessing-pipeline/src')
import numpy as np
import shutil
from skimage import io
from tqdm import tqdm
from atlas.NgSegmentMaker import NgConverter
from lib.sqlcontroller import SqlController
from lib.file_location import FileLocationManager
from lib.utilities_process import test_dir, SCALING_FACTOR
from lib.utilities_mask import place_image
import scipy
import cv2

In [11]:
animal = 'DK55'
sqlController = SqlController(animal)
fileLocationManager = FileLocationManager(animal)
INPUT = fileLocationManager.aligned_rotated_and_padded_thumbnail_mask
error = test_dir(animal, INPUT, downsample=True, same_size=True)
if len(error) > 0:
    print(error)
    sys.exit()

OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'shell')
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

files = sorted(os.listdir(INPUT))
len_files = len(files)
midpoint = len_files // 2
resolution = sqlController.scan_run.resolution
resolution = int(resolution * 1000 / SCALING_FACTOR)


In [49]:
volume = []
for file in tqdm(files):
    tif = io.imread(os.path.join(INPUT, file))
    # nseg,map,stat,position = cv2.connectedComponentsWithStats(tif)
    # maskid = np.argmax(stat[1:,-1])+1
    # tif = map[position[maskid].astype(int)]
    volume.append(tif)
ids = np.unique(volume)
ids = [(i,i) for i in ids]
volume = np.array(volume).astype('uint8')
volume = np.swapaxes(volume, 0, 2)


100%|██████████| 485/485 [00:28<00:00, 17.22it/s]


In [50]:
# fvolume = scipy.ndimage.gaussian_filter(volume,sigma=1.5)
fvolume = volume
fvolume = (fvolume!=0).astype('uint8')
fvolume = np.pad(fvolume,[[10,10],[10,10],[10,10]])
ng = NgConverter(fvolume, [resolution, resolution, 20000], offset=[0,0,0])
ng.create_neuroglancer_files(OUTPUT_DIR,ids)

Tasks:   0%|          | 0/16 [00:00<?, ?it/s]

No factors generated. Image Shape: (331, 22, 64, 1), Downsample Shape: [2048   22   64], Volume Shape: [2379   22  505], Bounds: Bbox([2048, 0, 0],[2379, 22, 64], dtype=int32)

Downloading:   0%|          | 0/32 [00:00<?, ?it/s]

Downloading: 100%|██████████| 6/6 [00:00<00:00, 67.78it/s]


Volume Bounds:  Bbox([0, 0, 0],[2379, 22, 505], dtype=int32)
Selected ROI:   Bbox([0, 0, 0],[2379, 22, 505], dtype=int32)


...

Downloading: 100%|██████████| 32/32 [00:00<00:00, 400.18it/s]


Downloading: 100%|██████████| 32/32 [00:00<00:00, 499.42it/s]

No factors generated. Image Shape: (2048, 22, 64, 1), Downsample Shape: [2048   22   64], Volume Shape: [2379   22  505], Bounds: Bbox([0, 0, 320],[2048, 22, 384], dtype=int32)


No factors generated. Image Shape: (2048, 22, 64, 1), Downsample Shape: [2048   22   64], Volume Shape: [2379   22  505], Bounds: Bbox([0, 0, 384],[2048, 22, 448], dtype=int32)


Downloading: 100%|██████████| 64/64 [00:00<00:00, 269.24it/s]


In [8]:
volume.shape

(2359, 1546, 485)